In [5]:
from typing import List

import numpy as np

from nn.value import Value
from nn.models import Perceptron
from nn.optimizer import AdamOptimizer

In [6]:
def make_perceptron_value(data: List[Value], weights: List[Value], bias: Value) -> Value:
    """Create and return a perceptron

    Parameters
    ----------
    data: list of Value objects (len should be >= 1)
        each entry represents a feature of x
    weights: list of Value objects (len should be >= 1)
        list of weights
    bias: Value
        represents a bias term
    """
    summands = map(lambda d, w: d * w, data, weights)
    
    linear_comb = next(summands)
    for summand in summands:
        linear_comb = linear_comb + summand

    logit = linear_comb + bias
    
    probability = logit.sigmoid()

    return probability

def make_binary_crossentropy_loss(prediction_values: List[Value], ground_truth: List[float]):
    loss = 0
    for p, g in zip(prediction_values, ground_truth):
        if np.isclose(g, 0):
            example_cost = (1 - p).log()
        elif np.isclose(g, 1):
            example_cost = p.log()
        loss = loss + example_cost
    loss = (-1) * loss
    return loss

This is an example that demonstrates how to creates loss function:

In [7]:
weights = [Value(0), Value(1)]
bias = Value(2)

data1 = [Value(3), Value(4)]
perceptron1 = make_perceptron_value(data1, weights, bias)

data2 = [Value(0), Value(0)]
perceptron2 = make_perceptron_value(data2, weights, bias)

loss = make_binary_crossentropy_loss([perceptron1, perceptron2], [1, 0])

This example demonstrates how to train a perceptron to behave like an AND gate:

In [8]:
X = [
    [0., 0.],
    [0., 1.],
    [1., 0.],
    [1., 1.],
    ]

Y = [0., 0., 0., 1.]

perceptron = Perceptron(no_weights=2)
adam_optimizer = AdamOptimizer(perceptron.parameters(), lr=0.5)

def simple_gradient_descent_iteration(model, X, Y, optimizer):
    optimizer.zero_grad()

    loss = sum(((model(x) - y)**2 for x, y in zip(X, Y)), 0.)

    loss.back_prop()

    optimizer.step()

    return loss.data

num_iterations = 100

for i in range(num_iterations):
    loss = simple_gradient_descent_iteration(perceptron, X, Y, adam_optimizer)
print(f"Final loss: {loss:.2}")
for x in X:
    pred_y = np.round(perceptron(x).data, 2)
    print(f"{int(x[0])} {int(x[1])}\t->\t{pred_y:.2f}")
# print("Final predictions =", *[ for x in X])

Final loss: 0.0033
0 0	->	0.00
0 1	->	0.03
1 0	->	0.03
1 1	->	0.97
